In [0]:
import fifeforspark
from fifeforspark.utils import create_example_data2
from fifeforspark.processors import PanelDataProcessor
from fifeforspark.lgb_modelers import LGBSurvivalModeler


In [0]:
# Import the data
data = fifeforspark.utils.create_example_data2()
test_intervals = 4

# Pass data through Panel Data Processor. We set test intervals to 4 to create the ability to test the model
# We set shuffle parts to 20 to reduce the amount of overhead since the dataset isn't that large.

processor = PanelDataProcessor(data=data, config = {'test_intervals': test_intervals}, shuffle_parts = 20)
processor.build_processed_data()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Expected a string or bytes object, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Time identifier column name not given; assumed to be second-leftmost column (period)
Individual identifier column name not given; assumed to be leftmost column (individual)
Now evaluating lead length: 1 of 4
Now evaluating lead length: 2 of 4
Now evaluating lead length: 3 of 4
Now evaluating lead length: 4 of 4

In [0]:
# Now, we build the model. You can pass parameters into the model that will be passed to lightgbm as well.
modeler = LGBSurvivalModeler(data=processor.data)
modeler.build_model(n_intervals=test_intervals)


In [0]:
# This part is unnecessary as it's equivalent to the default subset; however, we do it to show the ability to pass in a subset. 
min_val = modeler.data.select(modeler.data['_period']).agg({'_period': 'min'}).first()[0]

# We now predict the survival probabilities for our model.
evaluation_subset = modeler.data.select((modeler.data['_period'] == min_val).alias("subset")).select('subset')
predictions = modeler.predict(subset = evaluation_subset)
predictions.show()

+-------------+-------------+-------------+-------------+
probability_1|probability_2|probability_3|probability_4|
+-------------+-------------+-------------+-------------+
 0.99999976| 0.99999946| 0.9999969| 0.9999918|
 0.99999976| 0.99999946| 0.9999969| 0.9999918|
 0.99999976| 0.99999946| 0.9999969| 0.9999918|
 0.99999994| 0.9999996| 0.9999572| 0.9900357|
 0.9999999| 0.999974| 0.9979899| 0.8698845|
 0.99999976| 0.9999648| 0.99504757| 0.5477994|
 0.9997745| 0.91153044| 0.25220326| 0.075675376|
 0.97266775| 0.48291087| 0.090455994| 0.030645557|
 0.47298935| 0.119434| 0.022114275| 0.0022507266|
 0.67755824| 0.20009038| 0.025739422| 8.2631846E-4|
 0.37324396| 0.049055915| 0.0016174603| 3.719824E-5|
 0.13570897| 0.0127512| 1.5408089E-4| 3.9823694E-6|
 1.0| 0.9999999| 0.999995| 0.99996537|
 1.0| 0.99999845| 0.99990135| 0.99676013|
 0.9999999| 0.99996513| 0.98754954| 0.84781873|
 0.9999997| 0.99996346| 0.9476948| 0.8775395|
 0.99994254| 0.8067456| 0.38737434| 0.112851806|
 0.9998463| 0.82733154| 0.19605349| 0.04210605|
 0.9997968| 0.7038467| 0.3018352| 0.04587581|
 0.9992247| 0.7450975| 0.20660608| 0.039046265|
+-------------+-------------+-------------+-------------+
only showing top 20 rows

In [0]:
# We want to see how we did, so we print some evaluation metrics
evaluation = modeler.evaluate(subset = evaluation_subset)
evaluation

Out[11]:

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:
Lead Length,,,,,,,,
1,0.979218,0.904150,0.907620,3362,116,84,270,
2,0.977895,0.812850,0.818372,2991,145,110,586,
3,0.974619,0.719887,0.727557,2616,172,148,896,
4,0.972966,0.634334,0.631002,2246,172,166,1248,


In [0]:
# And finally, we forecast out from the last period of data.
forecasts = modeler.forecast()
forecasts